<a href="https://colab.research.google.com/github/ucfilho/Raianars_paper_one_revisited/blob/main/After_asked_reviews_one_2022/LSHADE/testLSHADE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import scipy.stats
import random

In [2]:
cols=30
dim = cols
NPAR1 = 3
MAX=np.repeat(100,dim)
MIN=np.repeat(-100,dim)
popsize = NPAR1
H = 6
kH = 0 # kH is used to work with H  
terminal = 0.1
p=0.1
SF=[];SCR=[];MF=[];MCR=[];Xarq=[]

In [3]:
# HappyCat function
# global mimima 0 at (-1,-1,...,-1)
# Not convex and non separable it is usually evaluated at xi =[(-600,600),..(-600,600)]

def fobj(x): 
    alpha=1. / 8
    s = sum(x**2) 
    return ((s - len(x))**2)**alpha + (s / 2 + sum(x)) / len(x) + 0.5 

In [4]:
def LShade(MAX,MIN, popsize,fobj,setTUNE,best,fbest,fitness,X,Xarq,FES):

  SF,SCR,MF,MCR,p, terminal, Narquive,H = setTUNE
  fmutant = []
  Xmutant = []
  fx=[]; fu=[] # to select the wij Lehmer Mean.
  # setTUNE = [SF,SCR,MF,MCR,p, terminal, Narquive,H] 
  dim = X.shape[1]
  #best_number = int(p*popsize)
  if (p*popsize<1):
    best_number=1
  else:
    best_number = int(p*popsize)
  justGoOUT=1

  if(justGoOUT==1):
    SCR =[];SF=[];
    for i in range(popsize):
      ri = random.randint(0,H-1) # line 7
      miF = MF[ri]
      miCR = MCR[ri]
      if(MCR[ri] == terminal):
        miCR = 0
      else:
        crossp = np.random.normal(miCR , 0.1)
      while (True):
        mut = scipy.stats.cauchy.rvs(loc=miF, scale=0.1)
        if (mut > 0):
          break

      ind = fitness.argsort()[range(best_number)] # find index of best p*popsize
      best_idx = random.choice(ind) # index of the best in p*popsize (random best)
      pbest = X[best_idx] # random best 
      idxs = [idx for idx in range(popsize) if idx != i]
      iused = np.random.choice(idxs, 1, replace = False)
      idxs = [idx for idx in range(popsize) if (idx != iused and idx != best_idx)]
      a  = X[iused]
      if(len(Xarq)>0):
        Xii = np.asarray(Xarq)
        Xnew = np.append(X[idxs,:],Xii,axis=0)
      else:
        Xnew=np.copy(X[idxs,:])

      idxs2 = [idx for idx in range(len(Xnew[:,0])) if 2 > 1]
      b  = Xnew[np.random.choice(idxs2, 1, replace = False)]

      mutant = X[i,:]+mut*(pbest-X[i,:]) + mut * (a - b)
      mutant = mutant.ravel()

      trial = np.copy(mutant)
    


      for j in range(dim):
        rdn = np.random.rand()
        if(rdn < crossp) : 
          trial[j] = mutant[j] # not necessary but to make clear (trial = np.copy(mutant))
        else:
          trial[j] = X[i,j]

      mutant  = np.copy(trial)
      
      for k in range(dim):
        if(mutant[k]>MAX[k]):
          mutant[k]=MAX[k]
        if(mutant[k]<MIN[k]):
          mutant[k]=MIN[k]

      Xmutant.append(mutant)

      FES = FES + 1
      fmut =fobj(mutant)
      fmutant.append(fmut)


    for i in range(popsize):
      if( fmutant[i]  <= fitness[i]):
        Xold = X[i,:] 
        X[i,:] = Xmutant[i]
        fitness[i]=fmutant[i]
        if(fbest >= fmutant[i]):
          best = Xmutant[i]
          fbest = fmutant[i]
      else:
        X[i,:] = X[i,:] # it is not necessary this line but just to remeber it
      if( fmutant[i]  < fitness[i]):
        if(len(Xarq[:,0])< Narquive):
          Xarq=np.append(Xarq,Xold,axis=0)
        else:
          ri = random.randint(0, Narquive-1) # deleting individuals if necessary line 19
          Xarq[ri,:] = Xold

        SCR.append(crossp)
        SF.append(mut)
        fx.append(fitness[i])
        fu.append(fmutant[i])
    # memory update algorithm 1:
    if (len(SCR)>0):
      deltaF = np.abs(np.asarray(fu)-np.asarray(fx))
      w = deltaF/sum(deltaF)
      if(MCR[kH] == terminal or max(SRC)==0):
        MCR[kH] = terminal
      else:
        y = np.asarray(SCR)
        MCR[kH] = np.dot(w,y**2)/np.dot(w,y)
      y = np.asarray(SF)
      MF[kH] = np.dot(w,y**2)/np.dot(w,y)
      kH = kH + 1
      if(kH > H): kH = 0
    else:
      MCR = MCR
      MF = MF
  
  setTUNE =[SF,SCR,MF,MCR,p, terminal, Narquive,H]
  

  y=fitness


  XY= np.c_[X,y] #concatena x e y em 2 colunas            
  XYsorted = XY[XY[:,-1].argsort()] #Ordena a partir da last col(Y) for all row
  X=XYsorted[:,0:dim]
  XY=XYsorted
  fitness = XYsorted[:,-1]  
  BEST=X[0,:]
  FOBEST=fitness[0]
  BEST_XY =np.append(BEST,FOBEST)
  
  return fitness,X,BEST,FOBEST,XY,BEST_XY,FES,Xarq,setTUNE


In [5]:

X=np.zeros((NPAR1,cols)) # X1 is Frannk population
for i in range(NPAR1):
  for j in range(cols):
    r=np.random.random()
    X[i,j]=r*(MAX[j]-MIN[j])+MIN[j]

FES = 0
X1 = np.copy(X)
fit1 = np.asarray([fobj(ind) for ind in X1])
FES = FES + NPAR1
best_idx = np.argmin(fit1)
best = X1[best_idx]
fbest = fit1[best_idx] 
BEST1,FOBEST1 = best, fbest

In [6]:
for i in range(H):
  MCR.append(0.5);MF.append(0.5) # line 3
Narquive = popsize
setTUNE =[SF,SCR,MF,MCR,p, terminal, Narquive,H]

In [7]:
for i in range(100):
  #print(i,X1.shape)
  fit1,X1,BEST1,FOBEST1,XY,BEST_XY,FES,Xarq,setTUNE= LShade(MAX,MIN, popsize,fobj,setTUNE,best,fbest,fit1,X1,Xarq,FES)
  print('i=',i,FOBEST1)

i= 0 1324.7558404455056
i= 1 1293.268884899187
i= 2 1259.4229355171276
i= 3 1132.8243453398852
i= 4 1132.8243453398852
i= 5 1027.558212798918
i= 6 1027.558212798918
i= 7 1015.1473860410373
i= 8 981.3921317065874
i= 9 967.8777076073575
i= 10 967.8777076073575
i= 11 919.3446893595435
i= 12 914.7655194559054
i= 13 911.463660568951
i= 14 911.463660568951
i= 15 911.463660568951
i= 16 908.2044435938332
i= 17 903.954858616181
i= 18 898.3210468967354
i= 19 893.6828748444049
i= 20 893.1363451799095
i= 21 889.7956492857862
i= 22 889.7956492857862
i= 23 889.3825468783236
i= 24 889.3825468783236
i= 25 889.1885138287502
i= 26 888.3718386271548
i= 27 887.8316429559437
i= 28 887.532913986464
i= 29 887.3249308286743
i= 30 886.8795202249
i= 31 886.7708444837169
i= 32 886.7708444837169
i= 33 886.5943759232068
i= 34 886.5943759232068
i= 35 886.3050511236746
i= 36 886.3050511236746
i= 37 886.3050511236746
i= 38 886.2714802811838
i= 39 886.2626595599686
i= 40 886.2249202846779
i= 41 886.2249202846779
i= 42

In [8]:
X1.shape

(3, 30)

In [9]:
fobj(X1[0,:])

883.5724784663535

In [10]:
fit1

array([883.57247847, 883.57438063, 883.57501288])